In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import os

# Load image in grayscale
def load_image(image_path):
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        raise ValueError("Image could not be loaded.")
    return img

# Manual 2D convolution implementation
def apply_convolution(image, kernel):
    img_height, img_width = image.shape
    kernel_height, kernel_width = kernel.shape
    pad_height, pad_width = kernel_height // 2, kernel_width // 2
    padded_img = np.pad(image, ((pad_height, pad_height), (pad_width, pad_width)), mode='constant', constant_values=0)
    output = np.zeros_like(image, dtype=np.float32)
    for i in range(img_height):
        for j in range(img_width):
            region = padded_img[i:i + kernel_height, j:j + kernel_width]
            output[i, j] = np.sum(region * kernel)
    output = np.clip(output, 0, 255)
    return output.astype(np.uint8)

# Define kernels
def get_kernels():
    sharpen_kernel = np.array([[0, -1, 0],
                             [-1, 5, -1],
                             [0, -1, 0]])
    blur_kernel = np.array([[1/9, 1/9, 1/9],
                           [1/9, 1/9, 1/9],
                           [1/9, 1/9, 1/9]])
    sobel_x = np.array([[-1, 0, 1],
                        [-2, 0, 2],
                        [-1, 0, 1]])
    sobel_y = np.array([[-1, -2, -1],
                        [0, 0, 0],
                        [1, 2, 1]])
    corner_kernel = sobel_x * sobel_y
    return {
        'sharpen': sharpen_kernel,
        'blur': blur_kernel,
        'corner': corner_kernel
    }

# Apply filters and compare with OpenCV
def process_image(image_path):
    img = load_image(image_path)
    kernels = get_kernels()
    results_manual = {}
    results_opencv = {}
    for name, kernel in kernels.items():
        results_manual[name] = apply_convolution(img, kernel)
        results_opencv[name] = cv2.filter2D(img, -1, kernel)
    return img, results_manual, results_opencv

# Add text to an image
def add_text_to_image(image, text):
    """
    Adds text to the top-left corner of an image using PIL.
    Args:
        image (numpy.ndarray): Grayscale image.
        text (str): Text to add (e.g., 'Original', 'Sharpen').
    Returns:
        PIL.Image: Image with text.
    """
    # Convert NumPy array to PIL Image
    img_pil = Image.fromarray(image)
    draw = ImageDraw.Draw(img_pil)
    # Use a default font (or specify a .ttf file if available)
    try:
        font = ImageFont.truetype("arial.ttf", 20)
    except:
        font = ImageFont.load_default()
    # Draw text in white with a black outline for visibility
    draw.text((10, 10), text, fill=255, font=font, stroke_width=1, stroke_fill=0)
    return img_pil

# Visualize results and save individual images
def visualize_results(original, manual_results, opencv_results):
    """
    Saves individual images with text and a comparison grid.
    Args:
        original (numpy.ndarray): Original image.
        manual_results (dict): Manual convolution results.
        opencv_results (dict): OpenCV convolution results.
    """
    # Ensure output directory exists
    os.makedirs('output', exist_ok=True)

    # Save individual images with text
    images_for_gif = []

    # Original image
    original_pil = add_text_to_image(original, "Original")
    original_pil.save('output/original.png')
    images_for_gif.append(original_pil)

    # Manual filter results
    for name in ['sharpen', 'blur', 'corner']:
        img = manual_results[name]
        img_pil = add_text_to_image(img, f"Manual {name.capitalize()}")
        img_pil.save(f'output/manual_{name}.png')
        images_for_gif.append(img_pil)

    # OpenCV filter results
    for name in ['sharpen', 'blur', 'corner']:
        img = opencv_results[name]
        img_pil = add_text_to_image(img, f"OpenCV {name.capitalize()}")
        img_pil.save(f'output/opencv_{name}.png')
        images_for_gif.append(img_pil)

    # Create comparison grid (manual vs. OpenCV)
    plt.figure(figsize=(15, 10))
    for i, name in enumerate(['sharpen', 'blur', 'corner'], 1):
        plt.subplot(3, 2, 2*i-1)
        plt.title(f'Manual {name.capitalize()}')
        plt.imshow(manual_results[name], cmap='gray')
        plt.axis('off')
        plt.subplot(3, 2, 2*i)
        plt.title(f'OpenCV {name.capitalize()}')
        plt.imshow(opencv_results[name], cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('output/comparison.png')
    plt.close()

    # Create GIF
    create_gif(images_for_gif)

# Create GIF from images
def create_gif(images):
    """
    Creates a GIF from a list of PIL images.
    Args:
        images (list): List of PIL.Image objects.
    """
    images[0].save(
        'output/convolution_results.gif',
        save_all=True,
        append_images=images[1:],
        duration=1000,  # 1 second per frame
        loop=0  # Loop indefinitely
    )

# Interactive interface with trackbars
def interactive_convolution(image_path):
    img = load_image(image_path)
    window_name = 'Interactive Convolution'
    cv2.namedWindow(window_name)
    kernel = np.ones((3, 3), dtype=np.float32)
    def update_kernel(*args):
        for i in range(3):
            for j in range(3):
                kernel[i, j] = cv2.getTrackbarPos(f'k[{i},{j}]', window_name) / 10.0
        result = cv2.filter2D(img, -1, kernel)
        cv2.imshow(window_name, result)
    for i in range(3):
        for j in range(3):
            cv2.createTrackbar(f'k[{i},{j}]', window_name, 10, 100, update_kernel)
    update_kernel()
    cv2.waitKey(0)
    cv2.destroyAllWindows()

# Main execution
if __name__ == '__main__':
    image_path = 'sample_image.jpg'
    original, manual_results, opencv_results = process_image(image_path)
    visualize_results(original, manual_results, opencv_results)
    interactive_convolution(image_path)